# Pembelajaran Mesin: Tugas Besar 1

Kaenova Mahendra Auditama  
IF-43-02  
1301190324  
  
Pada kodingan ini, saya mencoba untuk membuat sebuah model yang akan digunakan untuk membuat cluster pada suatu data. Data yang diberikan merupakan data ketertarikan pelanggan untuk membeli kendaraan baru. Pada akhirnya, saya menggunakan algoritma k-means karena kami diminta untuk membuat model dalam bentuk unsupervised learning

## Import Library yang Dibutuhkan untuk Data Preprocessing

In [1]:
import pandas as pd

## Preprocessing untuk K-Means Clustering
  
K-Means clustering hanya bisa dilakukan pada data-data dalam bentuk non-categorical ataupun ordinal. Sehingga saya harus membuang data-data yang berjenis seperti itu. Selain itu saya akan melihat outlier pada data setiap atributnya dan menghilangkannya menggunakan IQR (InterQuartileRange). Dengan perlakuan tersebut kita akan mendapatkan data yang cukup baik untuk dilakukan Clustering

In [6]:
df_main = pd.read_csv("../data/raw/kendaraan_train.csv")
df_main.sample(5)

,id,Jenis_Kelamin,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Umur_Kendaraan,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan,Tertarik
177794,177795,Pria,57.0,1.0,50.0,0.0,1-2 Tahun,Pernah,36020.0,26.0,22.0,0
228175,228176,Pria,31.0,1.0,6.0,1.0,< 1 Tahun,Tidak,NaN,160.0,232.0,0
154464,154465,Pria,59.0,1.0,46.0,1.0,1-2 Tahun,Tidak,24601.0,26.0,34.0,0
177951,177952,Pria,20.0,1.0,22.0,1.0,< 1 Tahun,Tidak,NaN,160.0,235.0,0
107379,107380,Pria,57.0,1.0,38.0,1.0,1-2 Tahun,Tidak,2630.0,26.0,60.0,0


### Lakukan drop data yang NaN

In [8]:
df_dropna = df_main.dropna(how="any")
df_dropna.sample(5)

,id,Jenis_Kelamin,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Umur_Kendaraan,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan,Tertarik
191049,191050,Pria,22.0,1.0,28.0,1.0,< 1 Tahun,Tidak,50347.0,152.0,118.0,0
275932,275933,Wanita,25.0,1.0,30.0,1.0,< 1 Tahun,Tidak,26507.0,152.0,197.0,0
282790,282791,Pria,71.0,1.0,18.0,1.0,1-2 Tahun,Tidak,37334.0,122.0,107.0,0
229173,229174,Wanita,52.0,1.0,28.0,0.0,1-2 Tahun,Pernah,34749.0,4.0,168.0,0
226056,226057,Pria,21.0,1.0,28.0,1.0,1-2 Tahun,Tidak,48535.0,124.0,128.0,0


### Penghapusan Data Kategorikal dan Set Index pada id
Hal ini dilakukan karena saya menggunakan algoritma K-Means yang tidak cocok pada data kategorikal dan ordinal

In [16]:
df_dropna_dropcategorical = df_dropna.drop(axis=1, columns=["Jenis_Kelamin", "Kode_Daerah", "SIM", "Umur_Kendaraan", "Sudah_Asuransi", "Kendaraan_Rusak", "Kanal_Penjualan"])
df_dropna_dropcategorical.set_index("id", inplace=True)
df_dropna_dropcategorical.sample(5)

,Umur,Premi,Lama_Berlangganan,Tertarik
id,,,,
273893,26.0,27553.0,115.0,0
118416,60.0,62642.0,194.0,0
257740,23.0,2630.0,62.0,1
89838,42.0,23594.0,18.0,0
26514,45.0,25087.0,26.0,1


### Memisahkan output (kolom tertarik) dan input

In [17]:
output_dropna_dropcategorical = df_dropna_dropcategorical["Tertarik"]
df_dropna_dropcategorical_dropoutput = df_dropna_dropcategorical.drop(axis=1, columns="Tertarik")
df_dropna_dropcategorical_dropoutput.sample(5)

,Umur,Premi,Lama_Berlangganan
id,,,
245855,32.0,21380.0,176.0
101477,33.0,45391.0,34.0
45870,21.0,34765.0,60.0
44705,24.0,37040.0,173.0
144317,66.0,25959.0,109.0


### Save Data Preprocessing ke csv

In [18]:
output_dropna_dropcategorical.to_csv("../data/processed/output_dropna_dropcategorical.csv")
df_dropna_dropcategorical_dropoutput.to_csv("../data/processed/input_dropna_dropcategorical.csv")

## Penentuan K value untuk K-Means
  
Pada algoritma K-Means, akan ada nilai k yang harus ditentukan untuk menentkan berapa jumlah cluster yang baik Pada metode ini saya menggunakan elbow analysis. Sehingga dari analisa metode elbow tersebut dapat ditentukan nilai k.